<a href="https://colab.research.google.com/github/tvelichkovt/TensorFlow/blob/master/1_TensorFlow_%3E_concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [234]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

print('TensorFlow ver is:', tf.__version__, '\nKeras ver is:', keras.__version__, '\nPandas ver is:', pd.__version__, '\nNumpy ver is:', np.__version__)

TensorFlow ver is: 2.1.0 
Keras ver is: 2.2.4-tf 
Pandas ver is: 0.25.3 
Numpy ver is: 1.17.5


In [0]:
# def

def f(x):
  return 2 * x + 1

f(5)

11

In [0]:
# tf.nn.relu with linear function

def f(n):
  return x + 1

@tf.function
def deep_net(n):
  return tf.nn.relu(f(n))

deep_net(tf.constant((1, 2, 3)))

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1.1542059, 1.4522265, 1.9370027],
       [1.9563732, 1.6440178, 1.3803433],
       [1.3337717, 1.2690653, 1.3602884]], dtype=float32)>

In [0]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
  return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])
# warm up
conv_layer(image); conv_fn(image)
print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number=10))
print("Function conv:", timeit.timeit(lambda: conv_fn(image), number=10))
print("Note how there's not much difference in performance for convolutions")


Eager conv: 1.5164551790003316
Function conv: 1.473996950997389
Note how there's not much difference in performance for convolutions


In [0]:
@tf.function
def square_if_positive(x):
  if x > 0:
    x = x * x
  else:
    x = 0
  return x


print('square_if_positive(2) = {}'.format(square_if_positive(tf.constant(2))))
print('square_if_positive(-2) = {}'.format(square_if_positive(tf.constant(-2))))

square_if_positive(2) = 4
square_if_positive(-2) = 0


In [0]:
@tf.function
def sum_even(items):
  s = 0
  for c in items:
    if c % 2 > 0:
      continue
    s += c
  return s


sum_even(tf.constant([10, 12, 15, 20]))

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [0]:
@tf.function
def fizzbuzz(n):
  for i in tf.range(n):
    if i % 3 == 0:
      tf.print('Fizz')
    elif i % 5 == 0:
      tf.print('Buzz')
    else:
      tf.print(i)

fizzbuzz(tf.constant(15))

Fizz
1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14


In [0]:
class CustomModel(tf.keras.models.Model):

  @tf.function
  def call(self, input_data):
    if tf.reduce_mean(input_data) > 0:
      return input_data
    else:
      return input_data // 2


model = CustomModel()

model(tf.constant([-2, -4]))

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([-1, -2], dtype=int32)>

In [0]:
v = tf.Variable(5)

@tf.function
def find_next_odd():
  v.assign(v + 1)
  if v % 2 == 0:
    v.assign(v + 1)


find_next_odd()
v

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=7>

In [0]:
@tf.function
def f(x):
  if x > 0:
    # Try setting a breakpoint here!
    # Example:
    #   import pdb
    #   pdb.set_trace()
    x = x + 1
  return x

tf.config.experimental_run_functions_eagerly(True)

# You can now set breakpoints and run the code in a debugger.
f(tf.constant(1))

tf.config.experimental_run_functions_eagerly(False)

In [0]:
# An in-graph training loop

x = tf.cast(x, tf.float32) / 255.0
y = tf.cast(y, tf.int64)
return x, y

def mnist_dataset():
  (x, y), _ = tf.keras.datasets.mnist.load_data()
  ds = tf.data.Dataset.from_tensor_slices((x, y))
  ds = ds.map(prepare_mnist_features_and_labels)
  ds = ds.take(20000).shuffle(20000).batch(100)
  return ds

train_dataset = mnist_dataset()

SyntaxError: ignored

In [0]:
model = tf.keras.Sequential((
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10)))
model.build()
optimizer = tf.keras.optimizers.Adam()

In [0]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()


def train_one_step(model, optimizer, x, y):
  with tf.GradientTape() as tape:
    logits = model(x)
    loss = compute_loss(y, logits)

  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  compute_accuracy(y, logits)
  return loss


@tf.function
def train(model, optimizer):
  train_ds = mnist_dataset()
  step = 0
  loss = 0.0
  accuracy = 0.0
  for x, y in train_ds:
    step += 1
    loss = train_one_step(model, optimizer, x, y)
    if step % 10 == 0:
      tf.print('Step', step, ': loss', loss, '; accuracy', compute_accuracy.result())
  return step, loss, accuracy

step, loss, accuracy = train(model, optimizer)
print('Final step', step, ': loss', loss, '; accuracy', compute_accuracy.result())

Step 10 : loss 1.7405858 ; accuracy 0.426
Step 20 : loss 1.17752218 ; accuracy 0.5735
Step 30 : loss 0.667794824 ; accuracy 0.647333324
Step 40 : loss 0.627128422 ; accuracy 0.68925
Step 50 : loss 0.484626412 ; accuracy 0.723
Step 60 : loss 0.415855795 ; accuracy 0.747666657
Step 70 : loss 0.652465343 ; accuracy 0.767142832
Step 80 : loss 0.329021364 ; accuracy 0.784375
Step 90 : loss 0.354959071 ; accuracy 0.796777785
Step 100 : loss 0.262325674 ; accuracy 0.8079
Step 110 : loss 0.408912957 ; accuracy 0.816
Step 120 : loss 0.270594686 ; accuracy 0.823083341
Step 130 : loss 0.286794484 ; accuracy 0.829461515
Step 140 : loss 0.240316913 ; accuracy 0.835857153
Step 150 : loss 0.296202958 ; accuracy 0.8414
Step 160 : loss 0.362588286 ; accuracy 0.8465
Step 170 : loss 0.311590225 ; accuracy 0.850882351
Step 180 : loss 0.222816125 ; accuracy 0.854777753
Step 190 : loss 0.392182469 ; accuracy 0.85794735
Step 200 : loss 0.26879409 ; accuracy 0.86175
Final step tf.Tensor(200, shape=(), dtype=i

In [0]:
# Batching

def square_if_positive(x):
  return [i ** 2 if i > 0 else i for i in x] # [-5, -4, -3, -2, -1, 0, 1, 4, 9, 16]

square_if_positive(range(-5, 5))

@tf.function
def square_if_positive_naive(x):
  result = tf.TensorArray(tf.int32, size=x.shape[0])
  for i in tf.range(x.shape[0]):
    if x[i] > 0:
      result = result.write(i, x[i] ** 2)
    else:
      result = result.write(i, x[i])
  return result.stack() #


square_if_positive_naive(tf.range(-5, 5))

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([-5, -4, -3, -2, -1,  0,  1,  4,  9, 16], dtype=int32)>

In [0]:
# tf.function can give you significant speedup over eager execution, at the cost of a slower first-time execution

import timeit

@tf.function
def f(x, y):
  return tf.matmul(x, y)

print(
    "First invocation:",
    timeit.timeit(lambda: f(tf.ones((10, 10)), tf.ones((10, 10))), number=1))

print(
    "Second invocation:",
    timeit.timeit(lambda: f(tf.ones((10, 10)), tf.ones((10, 10))), number=1))

First invocation: 0.06042392900053528
Second invocation: 0.0008281459995487239


In [0]:
# @tf.function may also re-trace when called with different non-tensor arguments:

@tf.function
def f(n):
  tf.print(n, '==== test TF ====')

f(9999)


9999 ==== test TF ====


In [0]:
# @tf.function decorator in a nested function:

def f():
  @tf.function
  def f():
    tf.print('==== test TF ====')
  f()

f()

==== test TF ====


In [0]:
# tf.where 1

def f(x):
  return tf.where(x > 0, x + 99, x)

f(tf.range(-1,1)) # [-1,  0]
f(tf.range(-2,2)) # [ -2,  -1,   0, 100]
f(tf.range(-3,3)) # [ -3,  -2,  -1,   0, 100, 101]
f(tf.range(-4,4)) # [ -4,  -3,  -2,  -1,   0, 100, 101, 102]

<tf.Tensor: shape=(8,), dtype=int32, numpy=array([ -4,  -3,  -2,  -1,   0, 100, 101, 102], dtype=int32)>

In [0]:
# tf.where 2

def f(x):
  return tf.where(x < 0, x + 99, x)

f(tf.range(-4,4)) # [95, 96, 97, 98,  0,  1,  2,  3]
f(tf.range(-2,2)) # [97, 98,  0,  1]

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([97, 98,  0,  1], dtype=int32)>

In [0]:
# tf.cast

x = tf.constant([1.8, 2.2], dtype=tf.float32)
tf.dtypes.cast(x, tf.int32)  # [1, 2], dtype=tf.int32


def prepare_mnist_features_and_labels(x, y):
  x = tf.cast(x, tf.float32) / 3.0
  y = tf.cast(y, tf.int32)
  return x, y

prepare_mnist_features_and_labels(2, 5.99) # 0.6666667 float32 & 5 int32

(<tf.Tensor: shape=(), dtype=float32, numpy=0.6666667>,
 <tf.Tensor: shape=(), dtype=int32, numpy=5>)

In [0]:
# tf.random.uniform

tf.random.uniform(shape=[4]) # [0.30896592, 0.5679636 , 0.09636641, 0.44236505]
tf.random.uniform(shape=[], minval=-1., maxval=0.) # -0.8583598
tf.random.uniform(shape=[], minval=5, maxval=10, dtype=tf.int64) #8
tf.random.uniform(shape=[], minval=5, maxval=10, dtype=tf.float16) #9.81
tf.random.uniform((3, 3)) # 3*3 array

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0.06874764, 0.76424766, 0.7879858 ],
       [0.07367384, 0.3478229 , 0.3734151 ],
       [0.46890664, 0.98679984, 0.3117875 ]], dtype=float32)>

In [0]:
# tf.matmul 2d + tf.nn.relu => Matrix product of two arrays with Relu -> a transformation that adds non-linearity.

a = tf.constant([1, 2, 3, 4, 5, 6], shape=[2, 3]) # 2-D tensor `a` matrix
b = tf.constant([7, 8, 9, 10, 11, 12], shape=[3, 2]) # 2-D tensor `b` matrix
c = tf.matmul(a, b) # # c1=1*7+2*9+3*11=58 ; c2=1*8+2*10+3*12=64 ; c3=4*7+5*9+6*11=139 ; c4=4*8+5*10+6*12=154 ==> matrix [[ 58,  64],[139, 154]] 
c


@tf.function
def simple_nn_layer(x, y):
  return tf.nn.relu(tf.matmul(x, y))


x = a #or tf.random.uniform((2, 2))
y = b #or tf.random.uniform((2, 2))

simple_nn_layer(x, y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

In [0]:
# tf.matmul 3d with tf.function

a = tf.constant(np.arange(1, 13, dtype=np.int32), shape=[2, 2, 3])
b = tf.constant(np.arange(13, 25, dtype=np.int32), shape=[2, 3, 2])
c = tf.matmul(a, b)
c

@tf.function
def matmul(x, y):
  return tf.matmul(x, y)
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

matmul(x, y)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0.2559033 , 0.13093121, 0.29653522],
       [1.2263331 , 0.6674587 , 1.6895535 ],
       [1.0543699 , 0.7248834 , 1.9126754 ]], dtype=float32)>

In [0]:
# TensorFlow program that uses relu

@tf.function
def relu(vector):
  return tf.nn.relu(vector))


vector = [0., -1.5, 1.5, 2.5, -3.5, -0.1]

relu(vector)

SyntaxError: ignored

In [0]:
# tf.constant

def linear_layer(x):
  return 2 * x + 1


@tf.function
def deep_net(x):
  return tf.nn.relu(linear_layer(x))


deep_net(tf.constant((1, 2, 3)))

tensor = tf.constant([1, 2, 3, 4, 5, 6, 7]) # vector constant 
tensor # [1, 2, 3, 4, 5, 6, 7]
tensor = tf.constant(-1.0, shape=[2, 3])
tensor # matrix 2/3
tensor = tf.constant((1, 2, 3))
tensor # [1, 2, 3]

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

In [0]:
add1 = tf.add(3, 5)
sess = tf.Session()
print(sess.run(a))
sess.close()

AttributeError: ignored

In [0]:
#https://adventuresinmachinelearning.com/python-tensorflow-tutorial/
#https://adventuresinmachinelearning.com/tensorflow-eager-tutorial/

const = tf.constant(2.0, name="const")
b = tf.Variable(2.0, name='b')
c = tf.Variable(1.0, name='c')

b.read_value()
c.read_value()

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [0]:
# now create some operations
d = tf.add(b, c, name='d')
e = tf.add(c, const, name='e')
a = tf.multiply(d, e, name='a')

In [0]:
# setup the variable initialisation
init_op = tf.global_variables_initializer()

AttributeError: ignored

In [0]:
# start the session
with tf.Session() as sess:
    # initialise the variables
    sess.run(init_op)
    # compute the output of the graph
    a_out = sess.run(a)
    print("Variable a is {}".format(a_out))

Variable a is 9.0


In [0]:
# create TensorFlow variables
b = tf.placeholder(tf.float32, [None, 1], name='b')

In [0]:
a_out = sess.run(a, feed_dict={b: np.arange(0, 10)[:, np.newaxis]})
#https://stackoverflow.com/questions/44151611/how-can-i-solve-this-runtimeerror-attempted-to-use-a-closed-session
#error

RuntimeError: ignored